In [ ]:
import pandas as pd
import plotly.express as px
import umap
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import StandardScaler

from functions.common_functions import *

In [2]:
# Import DF, separate training and virtual reactions 

df = pd.read_csv('../data/processed/CPA-virtual_database.csv')
df_training = df[df['Rxn_Type'] == 'Training']
df_training_rxns = df_training['rxn']
df_virtual = df[df['Rxn_Type'] == 'Virtual']
df_virtual = df_virtual[df_virtual['Catalyst_Ar_grp'] == 'L15']
df_virtual = df_virtual.reset_index(drop=True)

In [3]:
## Initial Processing ##

vt = VarianceThreshold()

parameters = df_virtual.iloc[:,9:] ## Set parameters to desired columns ##
thres = vt.fit(parameters)
parameters = thres.transform(parameters)

scaled_data = StandardScaler().fit_transform(parameters)

In [4]:
## Clustering Embedding ##

num_reactions = len(df_virtual)

reducer = umap.UMAP(
    random_state=25,
    min_dist=0,  # Default is 0.1
    n_neighbors=num_reactions/2,  # Default is 15
    n_components=10)

clustering_embedding = reducer.fit_transform(scaled_data)



In [5]:
## Elbow Plot ##

range_, inertia = get_clusters(clustering_embedding,100)

In [8]:
fig = px.scatter(x=range_, y=inertia,
                labels={'x':'# of Clusters', 'y':'Inertia'},
                template='simple_white'
                )

fig.update_traces(marker=dict(size=17,
                              line=dict(width=2,
                                        color='Black')),
                  selector=dict(mode='markers'))

fig.update_xaxes(mirror=True)
fig.update_yaxes(mirror=True)

fig.update_yaxes(range=[0,5000], row=1, col=1)

fig.update_layout(
    font_size=20,
)

fig.show()

In [5]:
## Get Cluster Labels ##

labels = k_cluster(clustering_embedding,50)
rxn_cluster_df = pd.DataFrame({'Reaction':df_virtual['rxn'],'cluster_label':labels})